In [1]:
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import numpy as np
import deepcut
import codecs

# Read Input

In [2]:
# Read Input and sequence number
read_input = [i.replace("\n", '') for i in open("input.txt", encoding="utf8")]

input_sentence = [r.split("::")[1] for r in  read_input]
input_sentence_num = [r.split("::")[0] for r in  read_input]

print(input_sentence[:10])
print(input_sentence_num[:10])

['เมื่อไหร่ลูกของฉันจะเป็นผู้เป็นคนอย่างเขาซักที', 'เขาโดนแตนต่อยที่ขา 4 ตัว', 'ดูผู้ชายคนนั้นสิ เขาแกะกุ้งให้แฟนกินด้วย ดีจังเลย', 'ทำไมเขาของไอ่ทุยถึงงอได้', 'ควายของเขานั้นฉลาดมาก', 'เขาของควายนั้นแข็งมาก', 'เขาแม้ว่าจะสูงเพียงใด ไม่ว่าใคร ถ้าตั้งใจเราจะพิชิตมันได้', 'ปัญหาแม้ว่าจะใหญ่เพียงใด ไม่ว่าใคร ถ้าตั้งใจเขาจะพิชิตมันได้', 'การปีนเขาเป็นกิจกรรมยามว่างที่ฉันทำเป็นประจำทุกอาทิตย์', 'เขาปีนกำแพงที่สูง เพื่อเข้าไปลักพาตัวเด็ก']
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


# Create word vector and Sentence vector

In [3]:
words = [[w for w in deepcut.tokenize(s) if w != ' '] for s in input_sentence]

In [4]:
# Extracted only the word between "เขา" 5 word
def split_from_kau(words):
    q = []
    flag_found_kau = False
    after_found = 0
    for w in words:
        q.append(w)
        if "เขา" in w:
            flag_found_kau = True
            after_found = len(q)
        elif flag_found_kau:
            if len(q) >= 21:
                break
        else:
            if len(q) > 10:
                q.pop(0)
    return q
# print(index_of_kau(words[0]))
words_split = [split_from_kau(w) for w in words]
print(words_split[-30:])
max_sentence_length = 21 # should not be larger than 21 word

[['รู้', 'ตัว', 'ดี', 'ว่า', 'ฉัน', 'ไม่', 'อาจ', 'จะ', 'ไป', 'เทียบ', 'เขา', 'ได้'], ['เขา', 'ไม่', 'รัก', 'เธอ', 'แล้ว', 'อย่าง', 'เสียใจ', 'ไป'], ['มอง', 'หา', 'ร้าน', 'อาหาร', 'ดี', 'ๆ', 'บรรยากาศ', 'วิว', 'เขา', 'กัน', 'อยู่', 'ใช่', 'ไหม'], ['โรงแรม', 'ใน', 'นครราชสีมา', 'ซึ่ง', 'อยู่', 'ใกล้', 'อุทยานแห่งชาติเขาใหญ่'], ['เขตรักษา', 'พันธุ์สัตว์ป่า', 'เขาอ่างฤๅไน', 'พบ', 'วัว', 'แดง', 'ใน', 'พื้นที่', 'หลาย', 'จุด'], ['เนิน', 'เขา', 'แห่ง', 'พระพุทธเจ้า', 'ถือ', 'เป็น', 'สถานที่', 'ท่องเที่ยว', 'แห่ง', 'ใหม่', 'ของ', 'เมืองซัปโปโร'], ['มี', 'ใคร', 'อยาก', 'เสีย', 'พลังงาน', 'ไป', 'กับ', 'การ', 'เดิน', 'ขึ้น', 'เขา', 'ที่', 'อยู่', 'ระหว่าง', 'ทาง'], ['สำหรับ', 'เส้นทาง', 'ลาด', 'ชัน', 'ขึ้น', 'เขา', 'หรือ', 'ขึ้น', 'เนิน', 'ต้อง', 'ใช้', 'ตำแหน่ง', 'เกียร์', 'ที่', 'เหมาะสม'], ['ที่', 'ราบ', 'เชิง', 'เขา', 'สี', 'เขียว', 'ขจี', 'ที่', 'ซึ่ง', 'เกิด', 'ขึ้น', 'ด้วย', 'ความ', 'ตั้งใจ', 'ของ', 'คุณเปิ้ล'], ['ตรึง', 'พื้นที่', 'เชิง', 'เขา', 'หลัง', 'เกิด', 'ปะทะ', 'กับ', 'กลุ่ม', 'ช

In [5]:
vocab = set([w for s in words_split for w in s])
pretrained_word_vec_file = open('cc.th.300.vec', 'r',encoding = 'utf-8-sig')
count = 0
vocab_vec = {}
for line in pretrained_word_vec_file:
    if count > 0:
        line = line.split()
        if(line[0] in vocab): 
            vocab_vec[line[0]] = line[1:]
    count = count + 1

In [6]:
# fill the initial with 0
word_vector_length = 300
word_vectors = np.zeros((len(words_split),max_sentence_length,word_vector_length))
sample_count = 0
for s in words_split:
    word_count = 0
    for w in s:
        try:
            word_vectors[sample_count,max_sentence_length-word_count-1,:] = vocab_vec[w]
            word_count = word_count+1
        except:
            pass
    sample_count = sample_count+1

# Create Dictlabel prepare to convert it back

In [7]:
dict_label = {"H":0,"P":1,"M":2}
dict_label_reverse = {v:k for k,v in dict_label.items()}
print(dict_label)
print(dict_label_reverse)

{'H': 0, 'P': 1, 'M': 2}
{0: 'H', 1: 'P', 2: 'M'}


# Load model

In [8]:
#Load model
model = load_model('model2.h5')

model.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 21, 300)]         0         
_________________________________________________________________
gru_4 (GRU)                  (None, 30)                29790     
_________________________________________________________________
dense_35 (Dense)             (None, 5)                 155       
_________________________________________________________________
dense_36 (Dense)             (None, 3)                 18        
Total params: 29,963
Trainable params: 29,963
Non-trainable params: 0
_________________________________________________________________


# Predict !!

In [9]:
y_pred = model.predict(word_vectors)
print(y_pred[:10])

[[9.5229901e-05 9.9977082e-01 1.3397919e-04]
 [2.4632862e-04 9.9959904e-01 1.5470682e-04]
 [1.8913430e-04 9.9974555e-01 6.5299326e-05]
 [9.9973232e-01 9.1851936e-05 1.7579696e-04]
 [9.9300051e-01 6.2333634e-03 7.6617114e-04]
 [9.9970478e-01 6.1147097e-05 2.3401437e-04]
 [3.1039606e-05 9.9831593e-01 1.6529926e-03]
 [1.9171915e-04 9.9974233e-01 6.5945213e-05]
 [4.3890649e-04 3.0997166e-04 9.9925107e-01]
 [4.6851453e-03 8.1262266e-04 9.9450219e-01]]


In [10]:
def find_index_of_max(l):
    i = (0,l[0])
    for y in enumerate(l):
        if i[1] < y[1]:
            i = y
    return i[0]

labels = [dict_label_reverse[find_index_of_max(pred)] for pred in y_pred]
print(labels)

['P', 'P', 'P', 'H', 'H', 'H', 'P', 'P', 'M', 'M', 'H', 'P', 'M', 'M', 'P', 'M', 'M', 'M', 'M', 'P', 'H', 'H', 'H', 'M', 'M', 'H', 'P', 'P', 'M', 'H', 'M', 'M', 'H', 'P', 'M', 'M', 'M', 'M', 'M', 'M', 'P', 'P', 'M', 'P', 'H', 'H', 'M', 'H', 'H', 'P', 'M', 'M', 'M', 'P', 'M', 'M', 'P', 'P', 'H', 'H', 'M', 'M', 'H', 'M', 'P', 'M', 'M', 'M', 'H', 'M', 'M', 'P', 'H', 'H', 'H', 'M', 'P', 'P', 'H', 'M', 'M', 'P', 'P', 'P', 'M', 'H', 'H', 'M', 'H', 'P', 'H', 'H', 'H', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'M', 'H', 'M', 'H', 'H', 'M', 'M', 'M', 'M', 'P', 'M', 'P', 'H', 'M', 'H', 'H', 'H', 'H', 'P', 'M', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'H', 'M', 'P', 'M', 'M', 'H', 'M', 'M', 'H', 'M', 'P', 'M', 'M', 'M', 'M', 'M', 'P', 'M', 'P', 'M',

# Write Results

In [11]:
with codecs.open("ans.txt", "w", "utf-8") as writer:
    for res in zip(input_sentence_num, labels):
        writer.write(res[0]+"::"+res[1]+"\n")